# 基于Transformers的多项选择

与机器阅读理解的不同之处在于输入是先给出context，再接上question和choice，有三个部分。context与question之间、choice最后有[SEP]分隔符，但question和choice可以只用空格分隔。此外有几个choice就构造几个输入，最后将[CLS]通过输出维度为1的全连接层，并拼接在一起做softmax取最大值，因为这说说是多项选择但答案只有一个，所以需要区分哪个选项最好。。。改进可能在trick上有一些修改，但本质还是这套

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

## Step1 导入相关包

In [2]:
import evaluate
import numpy as np
from datasets import DatasetDict # 数据集已经明确划分训练、验证和测试，就需要用DatasetDict（可以自己做一个dataset_dict.json）
from transformers import AutoTokenizer, AutoModelForMultipleChoice, TrainingArguments, Trainer

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
c3 = DatasetDict.load_from_disk("./c3/") # 必须是save_to_disk保存下来的数据集才能这么读取
c3 # 三个包含'id'、'context'、'question'、'choice'和'answer'键及对应值的字典

DatasetDict({
    test: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 1625
    })
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

In [4]:
c3["train"][:10] # 如果只有1个的话'id'、'question'和'answer'键对应的值会是字符串而不是列表

{'id': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'context': [['男：你今天晚上有时间吗?我们一起去看电影吧?', '女：你喜欢恐怖片和爱情片，但是我喜欢喜剧片，科幻片一般。所以……'],
  ['男：足球比赛是明天上午八点开始吧?', '女：因为天气不好，比赛改到后天下午三点了。'],
  ['女：今天下午的讨论会开得怎么样?', '男：我觉得发言的人太少了。'],
  ['男：我记得你以前很爱吃巧克力，最近怎么不吃了，是在减肥吗?', '女：是啊，我希望自己能瘦一点儿。'],
  ['女：过几天刘明就要从英国回来了。我还真有点儿想他了，记得那年他是刚过完中秋节走的。',
   '男：可不是嘛!自从我去日本留学，就再也没见过他，算一算都五年了。',
   '女：从2000年我们在学校第一次见面到现在已经快十年了。我还真想看看刘明变成什么样了!',
   '男：你还别说，刘明肯定跟英国绅士一样，也许还能带回来一个英国女朋友呢。'],
  ['男：好久不见了，最近忙什么呢?',
   '女：最近我们单位要搞一个现代艺术展览，正忙着准备呢。',
   '男：你们不是出版公司吗?为什么搞艺术展览?',
   '女：对啊，这次展览是我们出版的一套艺术丛书的重要宣传活动。'],
  ['男：会议结束后，你记得把空调和灯都关了。', '女：好的，我知道了，明天见。'],
  ['男：你出国读书的事定了吗?', '女：思前想后，还拿不定主意呢。'],
  ['男：这件衣服我要了，在哪儿交钱?', '女：前边右拐就有一个收银台，可以交现金，也可以刷卡。'],
  ['男：小李啊，你是我见过的最爱干净的学生。',
   '女：谢谢教授夸奖。不过，您是怎么看出来的?',
   '男：不管我叫你做什么，你总是推得干干净净。',
   '女：教授，我……']],
 'question': ['女的最喜欢哪种电影?',
  '根据对话，可以知道什么?',
  '关于这次讨论会，我们可以知道什么?',
  '女的为什么不吃巧克力了?',
  '现在大概是哪一年?',
  '女的的公司为什么要做现代艺术展览?',
  '他们最可能是什么关系?',
  '女的是什么意思?',
  '他们最可能在什么地方?',
  '教授认为小李怎么样?'],
 'choi

In [5]:
c3.pop("test") # "test"部分的'answer'是空的，所以需要先把它去掉

Dataset({
    features: ['id', 'context', 'question', 'choice', 'answer'],
    num_rows: 1625
})

In [6]:
c3

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer'],
        num_rows: 3816
    })
})

## Step3 数据集预处理

In [7]:
tokenizer = AutoTokenizer.from_pretrained("/data/PLM/chinese-macbert-base")
tokenizer

BertTokenizerFast(name_or_path='/data/PLM/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
def process_function(examples):
    # examples, dict, keys: ["context", "quesiton", "choice", "answer"]
    # examples, 1000
    context = []
    question_choice = []
    labels = []
    for idx in range(len(examples["context"])):
        ctx = "\n".join(examples["context"][idx]) # context是对话形式，所以需要先拼接一下
        question = examples["question"][idx]
        choices = examples["choice"][idx]
        for choice in choices:
            context.append(ctx) # 每个choice都对应相同的question和context，这样tokenizer的时候才能一一对应起来！！
            question_choice.append(question + " " + choice) # question和choice之间还是最好加一个空格以区分，也可以用其他的
        if len(choices) < 4:
            for _ in range(4 - len(choices)): # 不是每个问题都有4个选项，但最好还是填充成4个，因为最后要把[CLS]的结果拼接起来
                context.append(ctx)
                question_choice.append(question + " " + "以上都是") # 填充的选项倒是无所谓
        labels.append(choices.index(examples["answer"][idx])) # labels处理成一个数字，第几个选项
    tokenized_examples = tokenizer(context, question_choice, truncation="only_first", max_length=256, padding="max_length") # 每个键对应的值都是4000 * 256的嵌套列表,
    # 截断还是只截context吧！question和choice显然更关键！
    tokenized_examples = {k: [v[i: i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()} # 每个键对应的值都是1000 * 4 * 256的嵌套列表
    # tokenized_examples.items()要遍历也只有遍历键和值了！这样一来每个sample都包含4 * 256的input_ids等
    tokenized_examples["labels"] = labels # 笑死，最后再加上labels，正好“一一对应”
    return tokenized_examples
        

In [9]:
res = c3["train"].select(range(10)).map(process_function, batched=True)
res

Dataset({
    features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10
})

In [10]:
np.array(res["input_ids"]).shape # 转化为array就能看形状了

(10, 4, 256)

In [11]:
tokenized_c3 = c3.map(process_function, batched=True)
tokenized_c3

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 11869
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'choice', 'answer', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3816
    })
})

## Step4 创建模型

In [12]:
model = AutoModelForMultipleChoice.from_pretrained("/data/PLM/chinese-macbert-base")
model

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at /data/PLM/chinese-macbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForMultipleChoice(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

## Step5 创建评估函数

In [13]:
accuracy = evaluate.load("/data/daiyw/Compare/evaluate/metrics/accuracy")

def compute_metric(pred):
    predictions, labels = pred # 输出的第一个是预测，第二个才是标签！！！
    predictions = np.argmax(predictions, axis=-1) # 输出是array格式，必须用numpy才能处理
    return accuracy.compute(predictions=predictions, references=labels)

## Step6 配置训练参数

In [14]:
args = TrainingArguments(
    output_dir="./muliple_choice",
    per_device_train_batch_size=16, # 一个sample要承担原来4倍的工作量，所以batch size其实可以视为64！
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_steps=50,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True # 多了fp16的部分，这是因为batch size实质上扩大到原来的4倍，需要尽可能减少显存占用！
)

## Step7 创建训练器

In [15]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_c3["train"],
    eval_dataset=tokenized_c3["validation"],
    # data_collator=DefaultDataCollator(), # 其实不需要？最重要的批处理功能在args = TrainingArguments的batch参数设置中已经暗示了
    compute_metrics=compute_metric
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step8 模型训练

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.961100,0.925483,0.603249
2,0.678000,0.939685,0.632862
3,0.328800,1.296424,0.647013


TrainOutput(global_step=2226, training_loss=0.6738789636384016, metrics={'train_runtime': 774.8703, 'train_samples_per_second': 45.952, 'train_steps_per_second': 2.873, 'total_flos': 1.873702246273229e+16, 'train_loss': 0.6738789636384016, 'epoch': 3.0})

## Step9 模型预测

In [17]:
from typing import Any
import torch

class MultipleChoicePipeline:

    def __init__(self, model, tokenizer) -> None:
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device

    def preprocess(self, context, quesiton, choices):
        cs, qcs = [], []
        for choice in choices:
            cs.append(context)
            qcs.append(quesiton + " " + choice)
        return tokenizer(cs, qcs, truncation="only_first", max_length=256, return_tensors="pt") # 返回tensor格式

    def predict(self, inputs):
         # 本来v只有[num_choice, seq_length]的形状，这里要用unsqueeze加上batch_size的维度
        inputs = {k: v.unsqueeze(0).to(self.device) for k, v in inputs.items()} # 加载到GPU上居然可以只加载键的值？？自己做的时候还是全部加载吧
        return self.model(**inputs).logits

    def postprocess(self, logits, choices):
        predition = torch.argmax(logits, dim=-1).cpu().item()
        return choices[predition]

    def __call__(self, context, question, choices) -> Any: # 像pipeline这样声明对象后可以直接当函数用的类需要__call__函数
        inputs = self.preprocess(context, question, choices)
        logits = self.predict(inputs)
        result = self.postprocess(logits, choices)
        return result

In [18]:
pipe = MultipleChoicePipeline(model, tokenizer)

In [19]:
# 测试时num_choice可以不固定，甚至choice可以重复，这就是将[CLS]通过输出维度为1的全连接层，并拼接在一起做softmax取最大值进行分类的好处，不会受到choice个数的限制！
pipe(context = "小明在北京上班", question = "小明在哪里上班？", choices = ["北京", "上海", "河北", "海南"])

'北京'